# Assignment 2: Entity Resolution (Part 2)


## Objective

In Assignment 2 (Part 2), you will learn how to use Active Learning to address the entity resolution problem. After completing this assignment, you should be able to answer the following questions:

1. Why Active Learning?
2. How to implement uncertain sampling, a popular query strategy for Active Learning?
3. How to solve an ER problem using Active Learning?


## Active Learning

[Active learning](http://tiny.cc/al-wiki) is a certain type of ML algorithms that can train a high-quality ML model with small data-labeling cost. Its basic idea is quite easy to understand. Consider a typical supervised ML problem, which requires a (relatively) large training dataset. In the training dataset, there may be only a small number of data points that are beneficial to the trained ML model. In other words, labeling a small number of data points is enough to train a high-quality ML model. The goal of active learning is to help us to identify those data points. 


In this assignment, we will develop an Active Learning approach for Entity Resolution. The following figure shows the architecture of an entity resolution solution. It consists of four major steps. **I will provide you the source code for Steps 1, 2, 4. Your job is to implement Step 3.**  

<img src="img/arch.png", width=800/>

### Step 1. Read Data

Suppose we get a restaurant dataset `restaurant.csv`. The data has many duplicate restaurants.  For example, the first two rows shown below are duplicated (i.e., refer to the same real-world entity). You can check out all duplicate (matching) record pairs from `true_matches.json`. 

In [ ]:
import pandas as pd

df = pd.read_csv('restaurant.csv')
data = df.values.tolist()
print("(#Rows, #Cols) :", df.shape)
df.head(5)

### Step 2. Similar Pairs

We first use a similarity-join algorithm to generate similar pairs. 

Below is the code. After running the code, we get 678 similar pairs ordered by their similarity decreasingly.

In [ ]:
from a2_utils import *

data = df.values.tolist()
simpairs = simjoin(data)

print("Num of Pairs: ", len(data)*(len(data)-1)/2)
print("Num of Similar Pairs: ", len(simpairs))
print("The Most Similar Pair: ", simpairs[0])

We can see that `simjoin` helps us remove the number of pairs from 367653 to 678. But, there are still many non-matching pairs in `simpairs` (see below). 

In [ ]:
print(simpairs[-1])

Next, we will use active learning to train a classifier, and then use the classifier to classify each pair in `simpairs` as either "matching" or "nonmatching". 

### Step 3. Active Learning

Given a set of similar pairs, what you need to do next is to iteratively train a classifier to decide which pairs are truly matching. We are going to use [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) as our classifier. 

#### Initialization

At the beginning, all the pairs are unlabeled. To initialize a model, we first pick up ten pairs and then label each pair using  the `crowdsourcing()` function. You can assume that `crowdsourcing()` will ask a crowd worker (e.g., on Amazon Mechanical Turk) to label a pair. 


`crowdsourcing(pair)` is a function that simulates the use of crowdsourcing to label a pair
  
  - **Input:**	pair – A pair of records 

  - **Output:**	Boolean –  *True*: The pair of records are matching; *False*: The pair of records are NOT matching;

Please use the following code to do the initialization. 

In [ ]:
from a2_utils import crowdsourcing

# choose the most/least similar five pairs as initial training data
init_pairs = simpairs[:5] + simpairs[-5:]
matches = []
nonmatches = []
for pair in init_pairs:
    is_match = crowdsourcing(pair)
    if is_match == True:
        matches.append(pair)
    else:
        nonmatches.append(pair)
        
print("Number of matches: ", len(matches))
print("Number of nonmatches: ", len(nonmatches))

Here is the only code you need to write in this assignment.


In [ ]:
from a2_utils import featurize, crowdsourcing


labeled_pairs = matches + nonmatches
unlabeled_pairs = [p for p in simpairs if p not in labeled_pairs]
iter_num = 5

#<-- Write Your Code -->

**[Algorithm Description].**   Active learning has many [query strategies](http://tiny.cc/al-wiki-qs) to decide which data point should be labeled. You need to implement uncertain sampling. The algorithm trains an initial model on `labeled_pairs`. Then, it iteratively trains a model. At each iteration, it first applies the model to `unlabeled_pairs`, and makes a prediction on each unlabeled pair along with a probability, where the probability indicates the confidence of the prediction. After that, it selects the most uncertain pair (If there is still a tie, break it randomly),  and call the `crowdsroucing()` function to label the pair. After the pair is labeled, it updates `labeled_pairs` and `unlabeled_pairs`, and then retrain the model on `labeled_pairs`.

**[Input].** 
- `labeled_pairs`: 10 labeled pairs (by default)
- `unlabeled_pairs`: 668 unlabeled pairs (by default)
- `iter_num`: 5 (by default)

**[Output].** 
- `model`: A logistic regression model built by scikit-learn


### Step 4. Model Evaluation

After training an model, you can use the following code to evalute it.

In [ ]:
import json
from a2_utils import evaluate

            
sp_features = np.array([featurize(sp) for sp in simpairs])
label = model.predict(sp_features)
pair_label = zip(simpairs, label)

identified_matches = []
for pair, label in pair_label:
    if label == 1:
        identified_matches.append(pair)
        
precision, recall, fscore = evaluate(identified_matches)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", fscore)
   

## Submission

Complete the code in A2-2.ipynb, and submit it to the CourSys activity Assignment 2.